
- 此 script 用來搬運 mongo 內新聞的 html 資料到 minio，降低 mongo 的使用空間


In [1]:
import io
import json
import pymongo
from minio import Minio
from datetime import datetime

In [2]:
MONGO_URI = "mongodb://192.168.3.21:27019"
MINIO_URI = "192.168.3.21:27018"
mongo_client = pymongo.MongoClient(MONGO_URI)
minio_client = Minio(MINIO_URI, access_key="test", secret_key="testtest", secure=False)
TODAY_STR = datetime.today().strftime("%Y_%m_%d")
CHUNCK_SIZE = 20000 # depends on your device memory
# yahoo 5000 other 50000

# Write html data to Minio

In [3]:
def run(db):
    collection = db["html_data"]
    results = collection.find({})
    data_length = results.count()
    print("\t html_data length: {}".format(data_length))
    for i, start in enumerate(range(0, data_length, CHUNCK_SIZE)):
        results = collection.find({})
        data_jsons = [{"id": str(result["_id"]), "html_text": result["html_text"]} for result in results[start:start+CHUNCK_SIZE]]
        write_to_minio(data_jsons, db.name, i)

def run_industry(db, collection_name):
    collection = db[collection_name]
    industry_name = collection_name.split("_html_data")[0]
    results = collection.find({})
    data_length = results.count()
    print("\t {} length: {}".format(collection_name, data_length))
    for i, start in enumerate(range(0, data_length, CHUNCK_SIZE)):
        results = collection.find({})
        data_jsons = [{"id": str(result["_id"]), "html_text": result["html_text"]} for result in results[start:start+CHUNCK_SIZE]]
        write_to_minio(data_jsons, db.name, i, industry_name)

def write_to_minio(data_jsons, db_name, num, industry_name=None):
    data_str = json.dumps(data_jsons)
    data_bytes = data_str.encode('utf-8')
    data_stream = io.BytesIO(data_bytes)
    bucket_name = db_name.replace("_", "-")
    if not minio_client.bucket_exists(bucket_name):
        minio_client.make_bucket(bucket_name)
    if industry_name is None:
        minio_client.put_object(bucket_name, "{}_html_data_{}".format(TODAY_STR, num+1), data=data_stream , length=data_stream.getbuffer().nbytes)
    else:
        minio_client.put_object(bucket_name, "{}_{}_html_data_{}".format(TODAY_STR, industry_name, num+1), data=data_stream , length=data_stream.getbuffer().nbytes)

In [4]:
for db_name in mongo_client.list_database_names():
    db = mongo_client[db_name]
    if html_collection_name not in db.list_collection_names():
        continue
    print("start migrate {} html_data....".format(db_name))
#     run(db)
    try:
        run(db)
    except Exception as e:
        print("write {} html_data wrong: {}".format(db_name, e))
        continue

start migrate yahoo_data html_data....
	 html_data length: 249701


/home/terrylin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


# Industry html migrate

In [4]:
industry_db_name = "industry_data"
db = mongo_client[industry_db_name]

for collection_name in db.list_collection_names():
    if collection_name.endswith("html_data"):
        print("start migrate {}....".format(collection_name))
        run_industry(db, collection_name)

start migrate travel_html_data....
	 travel_html_data length: 692357


/home/terrylin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  from ipykernel import kernelapp as app


start migrate inv_trust_html_data....
	 inv_trust_html_data length: 145837
start migrate transport_html_data....
	 transport_html_data length: 18597


# Load back html data

In [6]:
def get_minio_data(bucket_name, file_name):
    stream = minio_client.get_object(bucket_name, file_name)
    s = stream.read()
    return s

In [ ]:
bucket_name = "yahoo-data"
filename = "{}_html_data_1".format(TODAY_STR)
minio_data = get_minio_data(bucket_name, filename)
data_jsons = json.loads(minio_data)

In [ ]:
len(data_jsons)